# Recipe2: Employer side Payroll Tax Offset 

This recipe is to introduce the usage of ` employer_payroll_offset` function from `payrolloffset.py`, when calling the function to conduct analysis upon employer side payroll tax reform

In [40]:
import taxcalcpayroll as tcp
from taxcalcpayroll.payrolloffset import employer_payroll_offset

import pandas as pd
import numpy as np

In [ ]:
recs = tcp.Records(data = 'puf.csv')
pol = tcp.Policy()
calc0 = tcp.Calculator(policy = pol, records = recs)

#### policy reform: increase the employer side FICA social security payroll tax rate to 0.072, for the year 2025 

In [42]:
reform1 = {'FICA_ss_trt_employer': {"2025": 0.072}}

### (1) total tax revenue after the reform (without offset)

In [ ]:
recs = tcp.Records(data = 'puf.csv')
pol1 = tcp.Policy()
pol1.implement_reform(reform1, print_warnings=True, raise_errors=True)
calc1 = tcp.Calculator(policy = pol1, records = recs)

calc1.advance_to_year(2025)
calc1.calc_all()
print("In 2025, combined tax revenue under reformed policy (without offset) is: ", calc1.weighted_total('combined') / 10**9)

In 2025, combined tax revenue under reformed policy (without offset) is:  $4053.78 Billion

### (2) total tax revenue after the reform, considering the offset

In [44]:
calc0.advance_to_year(2025)
calc0.calc_all()

### Function usage:
Input for offset function: baseline calculator (its policy and records object) and reform policy 

Output for offset function: returns a dataframe of output variables

In [ ]:
offdf = employer_payroll_offset(reform1, calc0, pol, recs, dump=False)    

print the result of calculated combined tax revenue

In [46]:
print("In 2025, combined tax revenue under reformed policy, after the offset, is: ", (offdf['combined'] * offdf['s006']).sum() / 10**9)

In 2025, combined tax revenue under reformed policy, after the offset, is:  4023.508250234729


In 2025, combined tax revenue under reformed policy, after the offset, is:  $4023.51 Billion

## Muti year usage

Comparison of total tax revenue calculation of baseline policy, reform policy (without offset), and reform policy considering offset

In [47]:
# setup some tool list to help printing result
dtable = pd.DataFrame([])
year_list = []
cur_list = []
reform_list = []
offset_list = []
base_ref_diff = []
ref_offset_diff = []
base_offset_diff = []

cur_sum = 0
reform_sum = 0
offset_sum = 0
base_ref_sum = 0
ref_offset_sum = 0
base_offset_sum = 0

In [ ]:
recs = tcp.Records(data = 'puf.csv')
pol = tcp.Policy()
calc0 = tcp.Calculator(policy = pol, records = recs)

recs = tcp.Records(data = 'puf.csv')
pol1 = tcp.Policy()
pol1.implement_reform(reform1, print_warnings=True, raise_errors=True)
calc1 = tcp.Calculator(policy = pol1, records = recs)

In [ ]:
# multi year 
for year in range(2024, 2034):   
    year_list.append(year)
    # baseline
    calc0.advance_to_year(year)
    calc0.calc_all()
    itax_rev0 = calc0.weighted_total('combined')
    itax_rev0 = (itax_rev0/10**9).round(2)
    cur_list.append(itax_rev0)
    cur_sum += itax_rev0
    
    # reform before offset
    calc1.advance_to_year(year)
    calc1.calc_all()
    itax_rev1 = calc1.weighted_total('combined')
    itax_rev1 = (itax_rev1/10**9).round(2)
    reform_list.append(itax_rev1)
    br_diff = itax_rev1 - itax_rev0
    base_ref_diff.append(br_diff)
    reform_sum += itax_rev0
    base_ref_sum += br_diff
    
    
    # offset
    offset_df = employer_payroll_offset(reform1, calc0, pol, recs, dump=False)
    itax_revbr = (offset_df['combined'] * offset_df['s006']).sum()
    itax_revbr = (itax_revbr/10**9).round(2)
    offset_list.append(itax_revbr)
    ro_diff = itax_revbr - itax_rev1
    ref_offset_diff.append(ro_diff)
    bo_diff = itax_revbr - itax_rev0
    base_offset_diff.append(bo_diff)
    offset_sum += itax_revbr
    ref_offset_sum += ro_diff
    base_offset_sum += bo_diff
    
    
year_list.append("10 year 2024-2033")  
cur_list.append(cur_sum)
reform_list.append(reform_sum)
base_ref_diff.append(base_ref_sum)
offset_list.append(offset_sum)
ref_offset_diff.append(ref_offset_sum)
base_offset_diff.append(base_offset_sum)


In [57]:
df = pd.DataFrame({'Year': year_list, 'Baseline Tototal Revenue': cur_list, 'Reform Tototal Revenue (no offset)': reform_list,
                   'Reform Tototal Revenue with offset': offset_list, "Diff: Reform - Baseline": base_ref_diff,
                   'Diff: Offset - Reform': ref_offset_diff, "Diff: Offset - Baseline": base_offset_diff
                  })

In [60]:
df = df.style.hide()

In [67]:
display(df)

,Year,Baseline Tototal Revenue,Reform Tototal Revenue (no offset),Reform Tototal Revenue with offset,Diff: Reform - Baseline,Diff: Offset - Reform,Diff: Offset - Baseline
0,2024,3757.61,3757.61,3757.61,0.00,0.00,0.00
1,2025,3952.99,4053.78,4023.51,100.79,-30.27,70.52
2,2026,4353.22,4458.41,4423.90,105.19,-34.51,70.68
3,2027,4543.83,4653.24,4617.23,109.41,-36.01,73.40
4,2028,4737.26,4850.96,4813.43,113.70,-37.53,76.17
5,2029,4935.73,5053.90,5014.79,118.17,-39.11,79.06
6,2030,5139.73,5262.54,5221.78,122.81,-40.76,82.05
7,2031,5348.20,5475.84,5433.37,127.64,-42.47,85.17
8,2032,5549.99,5682.67,5638.45,132.68,-44.22,88.46
9,2033,5781.67,5918.52,5872.84,136.85,-45.68,91.17
